# Equity Risk Premium Analysis
<p style="color:darkblue; "><b>A TurningBull Notebook</b>

In [20]:
library(httr)
library(jsonlite)
library(data.table)
library(quantmod)

In [ ]:
api.key <- read.csv('../.fmp_api.key', header = FALSE, stringsAsFactors = FALSE)[1,1]

In [ ]:
url <- "https://financialmodelingprep.com/stable/historical-price-eod/dividend-adjusted?symbol=SPY&from=1989-12-31&to=2025-02-28&apikey=2F6Gza19PPjluyAomWz16vmQGwH2DwvW"
response <- GET(url)

if (status_code(response) == 200) {
  spy <- content(response, "text")
  spy <- fromJSON(spy)
  spy <- data.table(spy)
  spy <- spy[, c("date", "adjClose")]
  spy <- spy[order(date)]
  } else {
  print("Error")
}

In [54]:
spy[, data := as.Date(date)]
spy.monthly <- spy[, .(adjClose = last(adjClose)), by = .(year(data), month(data))]
spy.monthly[, date := as.Date(paste(year, month, '01', sep = '-'))]
spy.monthly <- spy.monthly[, .(date, adjClose)]
spy.monthly[, rtn := adjClose / shift(adjClose) - 1]
head(spy.monthly)

In [40]:
getSymbols('DGS3MO', src = 'FRED')

In [53]:
t.bill <- data.table(date = index(DGS3MO), rate = DGS3MO[, 1])
t.bill <- t.bill[, .(rate = last(rate.DGS3MO)), by = .(year(date), month(date))]
t.bill[, date := as.Date(paste(year, month, '01', sep = '-'))]
t.bill <- t.bill[, .(date, rate)]
head(t.bill)

In [56]:
merge(spy.monthly, t.bill, by='date', all.x=TRUE)